In [1]:
import arcpy
import pandas as pd

def update_total_mins(feature_class, id_field, od_matrix_df, od_id_field, total_mins_field):
    # Create a dictionary from the OD matrix DataFrame for quick lookup
    total_mins_dict = pd.Series(od_matrix_df[total_mins_field].values, index=od_matrix_df[od_id_field]).to_dict()

    # Update cursor to iterate through rows in the feature class
    with arcpy.da.UpdateCursor(feature_class, [id_field, 'Total_Mins']) as cursor:
        for row in cursor:
            # Check if the id_field value exists in the dictionary
            if row[0] in total_mins_dict:
                # Update the 'Total_Mins' field with the value from the dictionary
                row[1] = total_mins_dict[row[0]]
                cursor.updateRow(row)

def add_and_update_field(shp_file, field_name, field_type):
    arcpy.AddField_management(shp_file, field_name, field_type)
    with arcpy.da.UpdateCursor(shp_file, [field_name]) as cursor:
        counter = 1
        for row in cursor:
            row[0] = counter
            cursor.updateRow(row)
            counter += 1

def remove_rows_if_condition(feature_class, condition_field_1, condition_field_2,condition_field_3):
    with arcpy.da.UpdateCursor(feature_class, [condition_field_1, condition_field_2,condition_field_3]) as cursor:
        for row in cursor:
            if row[0] > row[1] or row[2] < 0:
                cursor.deleteRow()

def get_common_hptripids(file_path):
    """
    Extract hptripid values from a shapefile.
    """
    hptripids = set()
    with arcpy.da.SearchCursor(file_path, ['hptripid']) as cursor:
        for row in cursor:
            hptripids.add(row[0])
    return hptripids

# Function to filter shapefile based on common hptripid values
def filter_shapefile(file_path, common_ids):
    # Create a query to select only the common hptripids
    query = "hptripid IN ('{}')".format("', '".join(common_ids))

    # Create a new shapefile with only the common hptripids
    filtered_file_path = file_path.replace('.shp', '_filtered.shp')
    arcpy.Select_analysis(file_path,filtered_file_path, query)

def Model(mode, mode_field):  
    arcpy.env.overwriteOutput = True

    # File paths and variables
    NetworkDataset_ND = rf"E:\Space-Time-Prism-Model\PwoD STP\MM_NetworkDataset_03092022.gdb\NetworkDataset\NetworkDataset_ND"
    origin_shp = rf"E:\Space-Time-Prism-Model\PwoD STP\Data\Data_Shapefiles\origin_{mode}.shp"
    destn_shp = rf"E:\Space-Time-Prism-Model\PwoD STP\Data\Data_Shapefiles\destn_{mode}.shp"
    od_cost_matrix_output = rf"E:\Space-Time-Prism-Model\PwoD STP\MyPwoDSTPProject\OD_Cost_Matrix_Results_{mode}.csv"

    # Add and update 'int_org_id' field in origin shapefile
    add_and_update_field(origin_shp, 'int_org_id', 'LONG')
    arcpy.AddField_management(origin_shp, "Total_Mins", "DOUBLE")
    # Add and update 'int_des_id' field in destination shapefile
    add_and_update_field(destn_shp, 'int_des_id', 'LONG')
    arcpy.AddField_management(destn_shp, "Total_Mins", "DOUBLE")

    # OD Cost Matrix Analysis Layer Name
    od_cost_matrix_layer_name = f"OD_Cost_Matrix_{mode}"

    # Delete the existing OD Cost Matrix Layer if it exists
    if arcpy.Exists(od_cost_matrix_layer_name):
        arcpy.management.Delete(od_cost_matrix_layer_name)

    # Create OD Cost Matrix Analysis Layer
    OD_Cost_Matrix_2 = arcpy.na.MakeODCostMatrixAnalysisLayer(
        network_data_source=NetworkDataset_ND,
        layer_name=od_cost_matrix_layer_name,
        travel_mode= mode,
        accumulate_attributes=["Miles",mode_field],
        ignore_invalid_locations="SKIP"
    )[0]

    # Set the search tolerance
    search_tolerance = "5000 Meters"
    search_criteria = [["BikePedAuto", "SHAPE"]]

    arcpy.na.AddLocations(OD_Cost_Matrix_2, "Origins", origin_shp, snap_to_position_along_network="SNAP", snap_offset="300 Meters", 
                          search_tolerance=search_tolerance, search_criteria=search_criteria)
    arcpy.na.AddLocations(OD_Cost_Matrix_2, "Destinations", destn_shp, snap_to_position_along_network="SNAP", snap_offset="300 Meters", 
                          search_tolerance=search_tolerance, search_criteria=search_criteria)
    arcpy.na.Solve(OD_Cost_Matrix_2, terminate_on_solve_error="CONTINUE")

    # Extract data from 'Lines' sublayer of OD Cost Matrix Analysis Layer
    lines_sublayer = arcpy.na.GetNAClassNames(OD_Cost_Matrix_2)["ODLines"]
    lines_layer = OD_Cost_Matrix_2.listLayers(lines_sublayer)[0]

    # Corrected file name with formatted string
    output_csv_file = f"OD_Cost_Matrix_Results_{mode}.csv"
    arcpy.conversion.TableToTable(lines_layer, fr"E:\Space-Time-Prism-Model\PwoD STP\MyPwoDSTPProject", output_csv_file)

    # Corrected file path to read the CSV
    od_matrix_df = pd.read_csv(fr"E:\Space-Time-Prism-Model\PwoD STP\MyPwoDSTPProject\{output_csv_file}")

    # Filter od_matrix_df for rows where OriginID equals DestinationID
    filtered_od_matrix_df = od_matrix_df[od_matrix_df['OriginID'] == od_matrix_df['DestinationID']]

    # Update 'Total_Mins' in origin and destination feature classes
    update_total_mins(origin_shp, 'int_org_id', filtered_od_matrix_df, 'OriginID', f"Total_{mode_field}")
    update_total_mins(destn_shp, 'int_des_id', filtered_od_matrix_df, 'DestinationID', f"Total_{mode_field}")

    # Remove rows where 'Total_Mins' is greater than 'Total_T'
    remove_rows_if_condition(origin_shp, 'Total_Mins', 'Total_F','Total_T')
    remove_rows_if_condition(destn_shp, 'Total_Mins', 'Total_F','Total_T')

    for file_path in [origin_shp,destn_shp]:
        arcpy.AddField_management(file_path, "Serv_Tbig", "DOUBLE")
        arcpy.AddField_management(file_path, "Serv_Tsm", "DOUBLE")
        arcpy.CalculateField_management(file_path, "Serv_Tbig", "!Total_Mins! + (!Total_T! - !Total_Mins!) / 2", "PYTHON3")
        arcpy.CalculateField_management(file_path, "Serv_Tsm", "!Total_T! / 2", "PYTHON3")

    # Extract hptripid values from both shapefiles
    origin_hptripids = get_common_hptripids(origin_shp)
    destn_hptripids = get_common_hptripids(destn_shp)
    # Identify common hptripid values
    common_hptripids = origin_hptripids.intersection(destn_hptripids)
    # Filter both shapefiles
    filter_shapefile(origin_shp, common_hptripids)
    filter_shapefile(destn_shp, common_hptripids)

    # Save the updated shapefiles
    origin_output_folder = rf"E:\Space-Time-Prism-Model\PwoD STP\Data\Data_Shapefiles"
    destn_output_folder = rf"E:\Space-Time-Prism-Model\PwoD STP\Data\Data_Shapefiles"
    arcpy.FeatureClassToShapefile_conversion([origin_shp], origin_output_folder)
    arcpy.FeatureClassToShapefile_conversion([destn_shp], destn_output_folder)

if __name__ == '__main__':
    with arcpy.EnvManager(scratchWorkspace=fr"E:\Space-Time-Prism-Model\PwoD STP\MyPwoDSTPProject\MyPwoDSTPProject.gdb", 
                          workspace=fr"E:\Space-Time-Prism-Model\PwoD STP\MyPwoDSTPProject\MyPwoDSTPProject.gdb"):
        modes=['Bike', 'Walk', 'Transit', "Drive"]
        mode_field_mapping = { 
            'Walk': 'Mins_PedOnly',
            'Bike': 'Mins_BikeOnly',
            'Drive': 'Mins_DriveOnly',
            'Transit': 'Mins_PedTransitOnly'
        }
        for mode in modes:  
            mode_field = mode_field_mapping[mode]
            Model(mode, mode_field)